In [1]:
#!/usr/bin/env python3

import pandas as pd


BLOCK_SIZE = 64


from base64 import b64encode, b64decode

def hex_to_base64(s):
    b64 = b64encode(bytes.fromhex(s)).decode()
    return b64

def base64_to_hex(b64):
    s2 = b64decode(b64.encode()).hex()
    return s2

def base64_encode(message_bytes):
    #message = "Python is fun"
    #message_bytes = message.encode('ascii')#converts into bytes
    base64_bytes = b64encode(message_bytes)#base64 encoding
    base64_message = base64_bytes.decode('utf-8')#bytes to ascii
    return base64_message

def base64_decode(string):
    base64_message = string
    base64_bytes = base64_message.encode('utf-8')
    message_bytes = b64decode(base64_bytes)#base64 decoding
    #message = message_bytes.decode('ascii')
    return message_bytes


def hex_to_bin(hex_str):
    return f'{int(hex_str, 16):0{len(4 * hex_str)}b}'

def pad(bin_str):
    return bin_str + '0' * ((BLOCK_SIZE - len(bin_str) % BLOCK_SIZE) % BLOCK_SIZE)

def fprint(text, variable):
    ""
    #print(f'{text:>22}: {variable}')

def split_block(block):
    return block[:len(block) // 2], block[len(block) // 2:]

def left_rotate(blocks, n_shifts):
    return [block[n_shifts:] + block[:n_shifts] for block in blocks]

def permute(block, table):
    return ''.join(block[i] for i in table)

def gen_subkeys(key):
    key_permutation_table = [
        56, 48, 40, 32, 24, 16, 8,
        0, 57, 49, 41, 33, 25, 17,
        9, 1, 58, 50, 42, 34, 26,
        18, 10, 2, 59, 51, 43, 35,
        62, 54, 46, 38, 30, 22, 14,
        6, 61, 53, 45, 37, 29, 21,
        13, 5, 60, 52, 44, 36, 28,
        20, 12, 4, 27, 19, 11, 3
    ]
    compression_permutation_table = [
        13, 16, 10, 23, 0, 4,
        2, 27, 14, 5, 20, 9,
        22, 18, 11, 3, 25, 7,
        15, 6, 26, 19, 12, 1,
        40, 51, 30, 36, 46, 54,
        29, 39, 50, 44, 32, 47,
        43, 48, 38, 55, 33, 52,
        45, 41, 49, 35, 28, 31
    ]
    left_rotate_order = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

    key_permutation = permute(key, key_permutation_table)

    fprint('KEY', key)
    fprint('KEY PERMUTATION', key_permutation)

    lk, rk = split_block(key_permutation)

    subkeys = []
    for n_shifts in left_rotate_order:
        lk, rk = left_rotate([lk, rk], n_shifts)
        compression_permutation = permute(lk + rk, compression_permutation_table)
        subkeys.append(compression_permutation)

    return subkeys


def xor(block_1, block_2):
    return f'{int(block_1, 2) ^ int(block_2, 2):0{len(block_1)}b}'


def s_box(block):
    s_box_table = [
        [
            [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
            [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
            [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
            [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
        ],
        [
            [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
            [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
            [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
            [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
        ],
        [
            [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
            [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
            [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
            [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
        ],
        [
            [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
            [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
            [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
            [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
        ],
        [
            [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
            [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
            [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
            [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
        ],
        [
            [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
            [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
            [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
            [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
        ],
        [
            [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
            [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
            [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
            [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
        ],
        [
            [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
            [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
            [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
            [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
        ]
    ]
    output = ''

    for i in range(8):
        sub_str = block[i * 6:i * 6 + 6]
        row = int(sub_str[0] + sub_str[-1], 2)
        column = int(sub_str[1:5], 2)
        output += f'{s_box_table[i][row][column]:04b}'

    return output


def round(input_block, subkey):
    expansion_permutation_table = [
        31, 0, 1, 2, 3, 4,
        3, 4, 5, 6, 7, 8,
        7, 8, 9, 10, 11, 12,
        11, 12, 13, 14, 15, 16,
        15, 16, 17, 18, 19, 20,
        19, 20, 21, 22, 23, 24,
        23, 24, 25, 26, 27, 28,
        27, 28, 29, 30, 31, 0
    ]
    p_box_table = [
        15, 6, 19, 20, 28, 11, 27, 16,
        0, 14, 22, 25, 4, 17, 30, 9,
        1, 7, 23, 13, 31, 26, 2, 8,
        18, 12, 29, 5, 21, 10, 3, 24
    ]

    l, r = split_block(input_block)
    expansion_permutation = permute(r, expansion_permutation_table)
    xor_1 = xor(expansion_permutation, subkey)
    s_box_output = s_box(xor_1)
    p_box = permute(s_box_output, p_box_table)
    xor_2 = xor(p_box, l)
    output = r + xor_2

    # fprint('INPUT', f'{l} {r}')
    # fprint('SUBKEY', subkey)
    # fprint('EXPANSION PERMUTATION', expansion_permutation)
    # fprint('XOR', xor_1)
    # fprint('S-BOX SUBSTITUTION', s_box_output)
    # fprint('P-BOX PERMUTATION', p_box)
    # fprint('XOR', xor_2)
    # fprint('SWAP', f'{r} {xor_2}')
    # fprint('OUTPUT', output)

    return output


def des(input_block, subkeys, crypt_type):
    res = []
    initial_permutation_table = [
        57, 49, 41, 33, 25, 17, 9, 1,
        59, 51, 43, 35, 27, 19, 11, 3,
        61, 53, 45, 37, 29, 21, 13, 5,
        63, 55, 47, 39, 31, 23, 15, 7,
        56, 48, 40, 32, 24, 16, 8, 0,
        58, 50, 42, 34, 26, 18, 10, 2,
        60, 52, 44, 36, 28, 20, 12, 4,
        62, 54, 46, 38, 30, 22, 14, 6
    ]
    final_permutation_table = [
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25,
        32, 0, 40, 8, 48, 16, 56, 24
    ]

    initial_permutation = permute(input_block, initial_permutation_table)

    #print()
    #print()
    #fprint('BLOCK', input_block)
    #fprint('INITIAL PERMUTATION', initial_permutation)

    if crypt_type == 'e':
        start = 0
        stop = 16
        step = 1
    else:
        start = 15
        stop = -1
        step = -1

    output = initial_permutation
    for i, j in enumerate(range(start, stop, step), 1):
        #print()
        #print(f'ROUND {i}:')
        output = round(output, subkeys[j])
        res.append(output)

    swap = output[BLOCK_SIZE // 2:] + output[:BLOCK_SIZE // 2]
    final_permutation = permute(swap, final_permutation_table)

    # print()
    fprint('SWAP', swap)
    fprint('FINAL PERMUTATION', final_permutation)

    return final_permutation,res


def crypt(mode, crypt_type, key_file, infile, iv_file=None):
    #bin_in_str = pad(hex_to_bin(infile.read()[:-1]))#modified
    bin_in_str = pad(hex_to_bin(infile))
    #subkeys = gen_subkeys(hex_to_bin(key_file.read()[:-1]))#modified
    subkeys = gen_subkeys(hex_to_bin(key_file))
    bin_out_str = ''

    if mode == 'ecb':
        for i in range(0, len(bin_in_str), BLOCK_SIZE):
            bin_out_str += des(bin_in_str[i:i + BLOCK_SIZE], subkeys, crypt_type)[0]

    else:
        each_rounds = {1:'',
                       2:'',
                       3:'',
                       4:'',
                       5:'', 
                       6:'', 
                       7:'', 
                       8:'', 
                       9:'', 
                       10:'', 
                       11:'', 
                       12:'', 
                       13:'', 
                       14:'', 
                       15:'', 
                       16:'',
                       17:''}
        
        #last_block = hex_to_bin(iv_file.read()[:-1])#modified
        last_block = hex_to_bin(iv_file)

        for i in range(0, len(bin_in_str), BLOCK_SIZE):
            block = bin_in_str[i:i + BLOCK_SIZE]

            if crypt_type == 'e':
                block = xor(block, last_block)

            temp_res = des(block, subkeys, crypt_type)
            output = temp_res[0]
            for i in range(1,17):
                each_rounds[i]+=str(temp_res[1][i-1])

            if crypt_type == 'e':
                last_block = output

            else:
                output = xor(output, last_block)
                last_block = block
            
            try:
                each_rounds[17]+=str(output)
            except UnboundLocalError:
                ""
                #print("\n\n\t*************1. EACH ROUND OUTPUT IS NOT CREATED***********\n\n")

            bin_out_str += output
    # try:
    #     for abc in each_rounds:
    #         print("round - ",abc,"===>",each_rounds[abc])
    # except UnboundLocalError:
    #     print("\n\n\t*************2. EACH ROUND OUTPUT IS NOT CREATED***********\n\n")

    #outfile.write(f'{int(bin_out_str, 2):0{len(bin_out_str) // 4}X}\n')
    return each_rounds

In [ ]:
if __name__ == '__main__':
    # def add_common_arguments(parser):
    #     crypt_group = parser.add_mutually_exclusive_group(required='True')
    #     crypt_group.add_argument('-e', action='store_const', dest='option', const='e', help='encrypt')
    #     crypt_group.add_argument('-d', action='store_const', dest='option', const='d', help='decrypt')
    #     parser.add_argument('key_file', type=argparse.FileType('r'),
    #                         help='text file to be used as key for encryption/decryption')
    #     parser.add_argument('infile', type=argparse.FileType('r'),
    #                         help='text file to be used as input for encryption/decryption')
    #     parser.add_argument('outfile', type=argparse.FileType('w'),
    #                         help='text file to be used as output for encryption/decryption')

    #     return parser


    # parser = argparse.ArgumentParser()
    # sub_parsers = parser.add_subparsers(dest='mode')
    # ecb_parser = add_common_arguments(
    #     sub_parsers.add_parser('ecb', help='use Electronic Codebook (ECB) encryption mode'))
    # cbc_parser = add_common_arguments(
    #     sub_parsers.add_parser('cbc', help='use Cipher Block Chaining (CBC) encryption mode'))
    # cbc_parser.add_argument('iv_file', type=argparse.FileType('r'),
    #                         help='text file to be used as IV for encryption/decryption')
    # args = parser.parse_args()
    
    #crypt(args.mode, args.option, args.key_file, args.infile, args.outfile, args.iv_file)
    def encrypt_des_cbc(key,infile,iv_file):
        return crypt('cbc', 'e', key, infile, iv_file)
    def decrypt_des_cbc(key,infile,iv_file):
        return crypt('cbc', 'd', key, infile, iv_file)
    key = "0F1571C947D9E859"
    iv_file = "FFFFFFFFFFFFFFFF"
    df = pd.read_csv("new.csv")
    final_res = []
    short_res = []
    for ii in range(6000):  
        plaintext = df.iloc[ii].text
        infile = "".join([hex(ord(i))[2:] for i in plaintext])
        # infile = pad(plaintext, DES.block_size)
        # infile = base64_to_hex(base64_encode(infile))
        #infile = "02468ACEECA8642002468ACEECA86420"
        enc_res = encrypt_des_cbc(key,infile,iv_file)
        # for i in enc_res:
        #     print(i,"--->",enc_res[i])
        # print("enc_res = ",enc_res)
        kys = enc_res.keys()
        # print("kys = ",kys)
        final_res.append([plaintext]+[hex(int(enc_res[q],2))[2:] for q in kys])
        short_res.append([plaintext]+[hex(int(enc_res[q],2))[2:2027] for q in kys])
    df1 = pd.DataFrame(final_res, columns=['plaintext','r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','r11','r12','r13','r14','r15','r16','r17'])
    df2 = pd.DataFrame(short_res, columns=['plaintext','r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','r11','r12','r13','r14','r15','r16','r17'])
    df1.to_csv("des_allrounds.csv")
    df2.to_csv("des_allrounds_2025.csv")

In [4]:
df1

,Unnamed: 0,plaintext,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17
0,0,"The Flesh and the Fiends, also known as Mania,...",ff11dd6f619225c109fd2eb820e6e4b95ac0d29e7d365b...,619225c149add86a20e6e4b9104c59c67d365b058300fb...,49add86a1ea1878c104c59c6ad13131f8300fb5293a531...,1ea1878ce5d780c4ad13131fe0bfce8d93a531aa589595...,e5d780c4a29251eee0bfce8dc73366095895955c4df13e...,a29251eee58ac90fc733660971a64c3d4df13ee843af78...,e58ac90f72f2482b71a64c3d3825758043af780f6e8d98...,72f2482b96690df2382575805de5c97a6e8d9858ec255b...,96690df2e202c4fb5de5c97a7c24019fec255b6e8c13f4...,e202c4fb583fd9297c24019fdaf2be038c13f4fb779f8e...,583fd929efa9324cdaf2be035756ab77779f8eabc00bac...,efa9324c5046ed5e5756ab77baa3c109c00bac6d009a71...,5046ed5e1e4974c0baa3c1091152135e009a7196a5b907...,1e4974c0a8da4c2a1152135e5015d22fa5b9075f37ada3...,a8da4c2a0902a7b05015d22f5a3bfa3a37ada330751a45...,902a7b0f12ee1595a3bfa3a6a01b7c8751a456d4acf1a7...,cd381891435e454e34ee04ebae6ec90d9b749a76a583db...
1,1,"Just because ""Pony Express"" is a western and t...",ff01fe5aed888d0d7c98d97fe7e576de3a035d8d97c737...,ed888d0df0a37167e7e576de313729c797c737f24a49dd...,f0a3716733c4f7c3313729c7b7bde83c4a49dd222296a4...,33c4f7c3041298a7b7bde83c553eea1a2296a4bf698776...,41298a7aae35493553eea1afc01cfcc69877684ddf285a...,aae35493c553a2cffc01cfcc7fda2770ddf285a6132785...,c553a2cf2f320a1b7fda2770d5f677ab1327856cf52516...,2f320a1bed938893d5f677abda3a4fc4f525166642476c...,ed938893899e097fda3a4fc41d7c1e4242476cadc44424...,899e097fd90a71fb1d7c1e4256dffd85c44424baa9c6f0...,d90a71fb67de8e9356dffd855f49a6c7a9c6f04339635f...,67de8e93292c71625f49a6c7d72efee039635fc1e7908f...,292c7162d0b6ccb0d72efee0d26b4caee7908ff4e53970...,d0b6ccb02ef619a1d26b4cae31ae8f62e53970189da69b...,2ef619a17c2759db31ae8f623af44ecc9da69bd72845d4...,7c2759dbf8b3105e3af44eccf74dbd0c2845d4066dceec...,3a33a1cbdff0cb5254c87f9fe4e47a6660137ed409c57d...
2,2,Triloquist starts way back in 1993 in Las Vega...,ff0163edee37345ca7ed7ad4fffe6c1f7509774daa8982...,ee37345cdbb27132fffe6c1f91ce5062aa8982542ace0f...,dbb27132f1345a2691ce50628eabb46d2ace0f7dd9ebff...,f1345a2674bff1388eabb46db15b1815d9ebffab1ddf4a...,74bff1388312898bb15b18156bdf5ab61ddf4a3d9995cc...,8312898b17a9ccff6bdf5ab696de28009995cc6d717388...,17a9ccff0a7645fb96de2800d7a57c7971738826437cf3...,a7645fb9bd547f2d7a57c7993336465437cf3f2c53f7f1...,9bd547f26eae4bf99333646501cacb0fc53f7f1ac815fb...,6eae4bf9c31be95c01cacb0ffc0bbd78c815fbd7dd39f8...,c31be95ccf6ddd53fc0bbd786f136147dd39f8bcdaf3ac...,cf6ddd53a8ead1a76f136147088e0d0fdaf3ac2ce4b55a...,a8ead1a764056ac0088e0d0f5dbc637ee4b55ad413b9d7...,64056ac0a5b6135b5dbc637e56510be213b9d7b852245c...,a5b6135ba7127a9d56510be275f6670852245cd27bcfce...,a7127a9de2b11cb475f667081e2c746b7bcfce276b3b38...,92e8870e3fd948d38969fc53e4bdad20f2fa2afc94d6e8...
3,3,"Noah Baumbach, the immensely talented writer, ...",ff21f4dcee014920989f16f92f48f0a531c55004b71ccb...,ee014920ccd0e28a2f48f0a5f28b9976b71ccb822ee11c...,ccd0e28abaab5a4df28b9976fb3072b52ee11c988eadfe...,baab5a4d6e517ee6fb3072b55764f3378eadfe4a95a0e2...,6e517ee60eea4ea85764f337beb25b3b95a0e2cc2f6965...,eea4ea8c347b5e3beb25b3b39a7b3672f6965744c22a51...,c347b5e3653c7ddc39a7b367997c2e6b4c22a512412c2b...,653c7ddce7239b49997c2e6b7fcf18d1412c2ba18893e0...,e7239b49497f0f1f7fcf18d1baf26b708893e0a0e6a3cb...,497f0f1f52c89bb8baf26b70b46c870be6a3cb6009e4e1...,52c89bb845045ec5b46c870b9b82e5c609e4e13d3c532d...,45045ec5f237a8b59b82e5c60a1ccb3e3c532db9adcfbb...,f237a8b5c19363f80a1ccb3e48ff10c5adcfbbf34044ca...,c19363f81e7c485c48ff10c55e4353c04044ca066a1690...,1e7c485c986027f35e4353c0313ae9846a169033124505...,986027f38986798f313ae984527e815612450549db32a7...,4f1b19c5862e26d38c711338f1b8590e6fd52d42d01563...
4,4,1968 was a remarkable year in the history of c...,ff00f55be0dd4dc371c3e1bbab33b72b6e058494f964cb...,e0dd4dc3c8521300ab33b72bf137349cf964cb356549c2...,c8521300aa444b27f137349ccb6b21436549c2275e38d7...,aa444b27549b8deacb6b21436705e9635e38d7527c8cd6...,549b8deaf3b235466705e963bbb658db7c8cd6a6e

In [5]:
df1.iloc[0]

Unnamed: 0                                                    0
plaintext     The Flesh and the Fiends, also known as Mania,...
r1            ff11dd6f619225c109fd2eb820e6e4b95ac0d29e7d365b...
r2            619225c149add86a20e6e4b9104c59c67d365b058300fb...
r3            49add86a1ea1878c104c59c6ad13131f8300fb5293a531...
r4            1ea1878ce5d780c4ad13131fe0bfce8d93a531aa589595...
r5            e5d780c4a29251eee0bfce8dc73366095895955c4df13e...
r6            a29251eee58ac90fc733660971a64c3d4df13ee843af78...
r7            e58ac90f72f2482b71a64c3d3825758043af780f6e8d98...
r8            72f2482b96690df2382575805de5c97a6e8d9858ec255b...
r9            96690df2e202c4fb5de5c97a7c24019fec255b6e8c13f4...
r10           e202c4fb583fd9297c24019fdaf2be038c13f4fb779f8e...
r11           583fd929efa9324cdaf2be035756ab77779f8eabc00bac...
r12           efa9324c5046ed5e5756ab77baa3c109c00bac6d009a71...
r13           5046ed5e1e4974c0baa3c1091152135e009a7196a5b907...
r14           1e4974c0a8da4c2a1152135e50